In [ ]:
pip install numpy pandas pikepdf matplotlib

In [32]:
import os
import re
import logging
import warnings
import pdfplumber
import pandas as pd

from typing import List, Dict, Any

warnings.filterwarnings("ignore")

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

DATE_PATTERN = re.compile(r'(?i)^\d{2}-[A-Za-z]{3}-\d{4}$')

AMOUNT_PATTERN = re.compile(r'^-?\d{1,3}(,\d{3})*(\.\d{1,2})?(Dr|Cr)?$', re.IGNORECASE)

def extract_text_plumber(pdf_path: str) -> str:
   
    full_text = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                full_text.append(page_text)
        return "\n".join(full_text)
    except Exception as e:
        logger.error(f"Error opening PDF {pdf_path}: {e}")
        return ""

def parse_statement_lines(text: str, debug: bool = False) -> List[Dict[str, str]]:
    
    results = []
    lines = text.splitlines()
    
    for idx, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue  
        
        if debug:
            print(f"[Line {idx}] {repr(line)}")
        
        tokens = line.split()
        if not tokens:
            continue
        
        if DATE_PATTERN.match(tokens[0]):
            date = tokens[0]
            
            trans_type = ""
            remainder = tokens[1:]
            if remainder and remainder[0] in ("T", "C"):
                trans_type = remainder[0]
                remainder = remainder[1:]
            
            amounts = []
            while remainder and AMOUNT_PATTERN.match(remainder[-1]):
                amounts.insert(0, remainder.pop())
            
            narration = " ".join(remainder)
           
            if len(amounts) == 2:
                transaction_amount = amounts[0]
                balance = amounts[1]
            elif len(amounts) == 1:
                transaction_amount = ""
                balance = amounts[0]
            else:
                transaction_amount = ""
                balance = ""
            
            # Build the record
            record = {
                "Date": date,
                "Trans_Type": trans_type,
                "Narration": narration,
                "Transaction_Amount": transaction_amount,
                "Balance": balance
            }
            results.append(record)
    
    return results

def export_to_excel(records: List[Dict[str, str]], pdf_path: str) -> str:
    
    if not records:
        logger.info("No records to export.")
        return ""
    
    df = pd.DataFrame(records)
    
    output_dir = "extracted_tables"
    os.makedirs(output_dir, exist_ok=True)
    
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    excel_path = os.path.join(output_dir, f"{base_name}_parsed.xlsx")
    
    df.to_excel(excel_path, index=False)
    logger.info(f"Parsed data exported to {excel_path}")
    return excel_path

def parse_bank_statement(pdf_path: str, debug: bool = False) -> List[Dict[str, str]]:
    
    text = extract_text_plumber(pdf_path)
    if not text.strip():
        logger.warning("No text extracted from PDF.")
        return []
    
    records = parse_statement_lines(text, debug=debug)
    logger.info(f"Total transactions parsed: {len(records)}")
    
    export_to_excel(records, pdf_path)
    
    return records

if __name__ == "__main__":
    pdf_path = r"C:\Users\DELL\Downloads\test6.pdf"
    parsed_data = parse_bank_statement(pdf_path, debug=False)
    
    if parsed_data:
        # Print first 5 records
        print("Sample Records:")
        for row in parsed_data[:5]:
            print(row)


pdf_path = r"C:\Users\DELL\Downloads\test3.pdf"
parsed_data = parse_bank_statement(pdf_path, debug=True)

2025-03-26 13:38:07,567 - INFO: Total transactions parsed: 0
2025-03-26 13:38:07,567 - INFO: No records to export.
2025-03-26 13:38:08,746 - INFO: Total transactions parsed: 70
2025-03-26 13:38:08,814 - INFO: Parsed data exported to extracted_tables\test3_parsed.xlsx


[Line 0] 'BANK NAME : PUNJAB AND SIND BANK 26-06-2023'
[Line 1] 'BRANCH NAME : JALANDHAR SME 13:15:47'
[Line 2] 'ADDRESS : 201 INDUSTRIAL AREA'
[Line 3] 'JALANDHAR'
[Line 4] 'CITY : JALANDHAR (PUNJAB)'
[Line 5] 'PIN CODE : 144001'
[Line 6] 'STATE : Punjab INDIA Page No: 1'
[Line 7] 'IFSC Code : PSIB0000697'
[Line 8] 'MICR Code : 144023019 Phone no:'
[Line 9] '0181-5018936'
[Line 10] 'REPORT PRINTED BY : MANPRIT KAUR Sol id : 0697'
[Line 11] '--------------------------------------------------------------------------------'
[Line 12] '--------------------------------------------------------------------------------'
[Line 13] '------------------------------'
[Line 14] 'Account No : 06971300000866 Nomination Registered :'
[Line 15] 'No'
[Line 16] 'A/C Name : M/S. KAMAL BRICKS INDUSTRY Nominee Name :'
[Line 17] 'Address : 560 MOTA SINGH NAGAR'
[Line 18] 'JALANDHAR'
[Line 19] 'City : JALANDHAR (PPin Code : 144001'
[Line 20] 'Tel No. : Sanction Limit :'
[Line 21] '3000000'
[Line 22] 'A/C Hold

In [29]:
pdf_path = r"C:\Users\DELL\Downloads\test3.pdf"
parsed_data = parse_bank_statement(pdf_path, debug=True)

2025-03-26 13:36:15,217 - INFO: Total transactions parsed: 70
2025-03-26 13:36:15,282 - INFO: Parsed data exported to extracted_tables\test3_parsed.xlsx


[Line 0] 'BANK NAME : PUNJAB AND SIND BANK 26-06-2023'
[Line 1] 'BRANCH NAME : JALANDHAR SME 13:15:47'
[Line 2] 'ADDRESS : 201 INDUSTRIAL AREA'
[Line 3] 'JALANDHAR'
[Line 4] 'CITY : JALANDHAR (PUNJAB)'
[Line 5] 'PIN CODE : 144001'
[Line 6] 'STATE : Punjab INDIA Page No: 1'
[Line 7] 'IFSC Code : PSIB0000697'
[Line 8] 'MICR Code : 144023019 Phone no:'
[Line 9] '0181-5018936'
[Line 10] 'REPORT PRINTED BY : MANPRIT KAUR Sol id : 0697'
[Line 11] '--------------------------------------------------------------------------------'
[Line 12] '--------------------------------------------------------------------------------'
[Line 13] '------------------------------'
[Line 14] 'Account No : 06971300000866 Nomination Registered :'
[Line 15] 'No'
[Line 16] 'A/C Name : M/S. KAMAL BRICKS INDUSTRY Nominee Name :'
[Line 17] 'Address : 560 MOTA SINGH NAGAR'
[Line 18] 'JALANDHAR'
[Line 19] 'City : JALANDHAR (PPin Code : 144001'
[Line 20] 'Tel No. : Sanction Limit :'
[Line 21] '3000000'
[Line 22] 'A/C Hold